In [1]:
import sys, git
sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from python.yaml_cfg import Config

In [2]:
doc = """
label: "simple"

file: "DoubleMu_GMTIso_ID"
filepath: "/eos/uscms/store/user/ekoenig/trigger/"
outfile: "effi_{file}_{label}_{tag}.root"

branch: "gmtTkMuon"
eta_range: [-2.4, 2.4]
pt_range: [2.0, 100.0]

total: -1

matched_delta_r: 0.1

gen_variables:
    pt: "partPt"
    eta: "partEta"
    phi: "partPhi"


gen_selection:
    stat: "partStat==1"
    etacut: "(partEta > {eta_range[0]}) & (partEta < {eta_range[1]})"

l1_variables:
    pt: "{branch}Pt"
    eta: "{branch}Eta"
    phi: "{branch}Phi"
    nstubs: "{branch}NStubs"
"""

In [3]:
cfg = Config.from_str(doc)

In [35]:
from string import Formatter

def get_replace(string): 
    return [fn for _, fn, _, _ in formatter.parse(string) if fn is not None]
def replace_available(string : str, **namespace):
    fmt_replace = {
        key: namespace.get(key, '{'+key+'}')
        for key in get_replace(string)
    }      
    return string.format(**fmt_replace)

def fill_replace(variable, **namespace):
    if isinstance(variable, str):
        variable = replace_available(variable, **namespace)

    elif isinstance(variable, dict):
        new_namespace = dict(namespace, **variable)
        variable = { key: fill_replace(value, **new_namespace) for key, value in variable.items() }

    elif isinstance(variable, list):
        variable = [ fill_replace(value) for value in variable]
        
    return variable

fill_replace(vars(cfg), **vars(cfg))

KeyError: 'eta_range'

In [112]:
from string import Formatter
 
test_str = "testing formating {notgiven} with a {given[0]} and {alsonotgiven}"

In [113]:
fields = [fn.replace(" ","") for _, fn, _, _ in Formatter().parse(test_str) if fn is not None]
fields

['notgiven', 'given[0]', 'alsonotgiven']

In [114]:
namespace = dict( given = ['(first element)'], another="wasnt needed" )

In [115]:
new_namespace = dict(namespace, notgiven='{notgiven}', alsonotgiven=r'{alsonotgiven[1]}')

In [116]:
class FormatDict(dict):
    def __missing__(self, key):
        return "{" + key + "}"

In [117]:
Formatter().vformat(test_str, (), FormatDict(**namespace))

'testing formating {notgiven} with a (first element) and {alsonotgiven}'

In [77]:
test_str.format(**namespace)

KeyError: 'notgiven'

In [78]:
test_str.format(**new_namespace)

'testing formating {notgiven} with a (first element) and a'

In [45]:
formatter.get_field('notgiven[0]')

TypeError: get_field() missing 2 required positional arguments: 'args' and 'kwargs'